In [1]:
# 预测全家桶与机器学习神器

In [2]:
## Attrition

In [3]:
### Attrition Catboost

In [4]:
import pandas as pd

In [26]:
trainset=pd.read_csv('../../resource/BI/train.csv',index_col=0)
testset=pd.read_csv('../../resource/BI/test.csv',index_col=0)

In [27]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
trainset['Attrition']=trainset['Attrition'].map(lambda x:1 if x=='Yes' else 0)


In [28]:
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [29]:
# 去掉没用的列 员工号码，标准工时（=80）
train = trainset.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = testset.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [30]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

In [31]:
import catboost as cb
import numpy as np
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [33]:
model = cb.CatBoostClassifier(iterations=1000, 
                              depth=7, 
                              learning_rate=0.01, 
                              loss_function='Logloss', 
                              eval_metric='AUC',
                              logging_level='Verbose', 
                              metric_period=50
                             )

In [34]:
# 得到分类特征的列号
categorical_features_indices = []
for i in range(len(X_train.columns)):
    if X_train.columns.values[i] in attr:
        categorical_features_indices.append(i)
print(categorical_features_indices)

[0, 1, 3, 5, 6, 9, 13, 15, 19, 20]


In [35]:
model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=categorical_features_indices)

0:	test: 0.6390374	best: 0.6390374 (0)	total: 2.7ms	remaining: 2.69s
50:	test: 0.7893703	best: 0.7893703 (50)	total: 159ms	remaining: 2.95s
100:	test: 0.7950453	best: 0.7950453 (100)	total: 329ms	remaining: 2.93s
150:	test: 0.8011568	best: 0.8011568 (150)	total: 502ms	remaining: 2.82s
200:	test: 0.7958092	best: 0.8011568 (150)	total: 695ms	remaining: 2.76s
250:	test: 0.8023573	best: 0.8023573 (250)	total: 889ms	remaining: 2.65s
300:	test: 0.8034487	best: 0.8034487 (300)	total: 1.06s	remaining: 2.45s
350:	test: 0.8046491	best: 0.8046491 (350)	total: 1.24s	remaining: 2.29s
400:	test: 0.8030121	best: 0.8046491 (350)	total: 1.43s	remaining: 2.13s
450:	test: 0.8054131	best: 0.8054131 (450)	total: 1.64s	remaining: 1.99s
500:	test: 0.8067227	best: 0.8067227 (500)	total: 1.86s	remaining: 1.85s
550:	test: 0.8062862	best: 0.8067227 (500)	total: 2.04s	remaining: 1.66s
600:	test: 0.8056313	best: 0.8067227 (500)	total: 2.23s	remaining: 1.48s
650:	test: 0.8053039	best: 0.8067227 (500)	total: 2.42s	r

In [36]:
#model = cb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test)
#predict = model.predict_proba(test)
#print(predict)
test['Attrition']=predict

In [37]:
## 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test[['Attrition']].to_csv('submit_cb.csv')

In [38]:
## Attrition GBDT

In [39]:
import pandas as pd

In [40]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=trainset['Attrition'].map(lambda x:1 if x=='Yes' else 0)

In [41]:
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [42]:
# 去掉没用的列 员工号码，标准工时（=80）
train = trainset.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = testset.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [43]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('temp.csv')
#print(train)

In [44]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [45]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [46]:
# 采用回归算法，可以得到更好的AUC结果
model = GradientBoostingRegressor(random_state=10)
model.fit(X_train, y_train)
predict = model.predict(test)
print(predict)
test['Attrition']=predict
#print(predict)

[ 1.01678366e-01  4.47445681e-02  1.42121556e-01  7.57204880e-02
  7.03200210e-01  3.70055651e-01  3.74376235e-01  4.37242139e-02
 -1.15634448e-01  2.97987772e-01  1.20779518e-01  1.09366950e-01
  1.07679394e-01  7.66600582e-01  7.51844882e-02 -4.76111832e-02
  4.28285053e-02 -1.09673242e-02  1.59486732e-01  9.62417811e-02
  6.47325348e-01  1.10040397e-01 -9.94554567e-03 -4.55439809e-02
  2.68815031e-01  3.37430996e-01  7.35105859e-02  1.06048931e-01
  5.50770434e-01  4.15464265e-02  1.79826162e-01  1.01207422e-03
  1.58722155e-01  1.58671176e-01  8.62998751e-02  2.67636183e-02
  1.24005756e-01  9.94003577e-02  8.73177129e-02  1.59141438e-03
  3.02853331e-02 -5.02317582e-03 -2.69498786e-03  8.75329962e-03
  3.69313030e-02  5.03418977e-01  2.62794122e-01 -1.16559089e-02
  8.28838190e-01  4.78927949e-01  2.30745577e-01  3.61855221e-01
  1.97193403e-01  2.08480315e-01  4.27257886e-01  7.31510619e-02
 -4.10414290e-02  1.41280282e-01 -5.16541193e-02  4.63709471e-01
  8.91047041e-03  1.57167

In [47]:
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test[['Attrition']].to_csv('submit_gbdt.csv')
print('submit_gbdt.csv saved')

submit_gbdt.csv saved


In [48]:
### Attrition LGB Onehot

In [49]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

In [51]:
train=pd.read_csv('../../resource/BI/train.csv',index_col=0)
test=pd.read_csv('../../resource/BI/test.csv',index_col=0)
#print(train['Attrition'].value_counts())

In [52]:
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

# 查看数据是否有空值
#print(train.isna().sum())

In [53]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [54]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']

In [55]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [56]:
param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', #
                         'eval_metric' : 'auc',
                         'eta' : 0.01,
                         'max_depth' : 15,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,
        }

In [57]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [58]:
# 使用DictVectorizer对离散值进行编码
dvec = DictVectorizer(sparse=False)
X_train = dvec.fit_transform(X_train.to_dict(orient='record'))
X_valid = dvec.transform(X_valid.to_dict(orient='record'))
temp_test = dvec.transform(test.to_dict(orient='record'))

/Users/du/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/du/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  after removing the cwd from sys.path.
/Users/du/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one o

In [59]:
# 进行DMatrix转换
train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)
test_data = xgb.DMatrix(temp_test)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test_data)
test['Attrition']=predict
print(predict)

[00:18:55] WARNING: /private/var/folders/h4/7cr1cmpn7v5b3x20_9wz8m740000gn/T/pip-req-build-nxocilc1/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:573: 
Parameters: { "boosting_type", "subsample_freq" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.83432	valid-auc:0.66839
[25]	train-auc:0.96482	valid-auc:0.74855
[50]	train-auc:0.97567	valid-auc:0.76416
[75]	train-auc:0.98638	valid-auc:0.77900
[100]	train-auc:0.98983	valid-auc:0.77715
[125]	train-auc:0.99280	valid-auc:0.78326
[150]	train-auc:0.99485	valid-auc:0.79395
[175]	train-auc:0.99713	valid-auc:0.79810
[200]	train-auc:0.99832	valid-auc:0.79996
[225]	train-auc:0.99893	valid-auc:0.80367
[250]	train-auc:0.99942	valid-auc:0.80541
[275]	train-auc:0.99969	valid-auc:0.80661
[300]	train-auc:0.99982	valid

In [60]:
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test[['Attrition']].to_csv('submit_xgb.csv')

In [61]:
### Attrition LGB

In [62]:
import pandas as pd

In [63]:
train=pd.read_csv('../../resource/BI/train.csv',index_col=0)
test=pd.read_csv('../../resource/BI/test.csv',index_col=0)

In [64]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [65]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [66]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('temp.csv')
#print(train)

In [70]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

OSError: dlopen(/Users/du/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/du/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [68]:
# param = {
#     'num_leaves':41,
#     'boosting_type': 'gbdt',
#     'objective':'binary',
#     'max_depth':15,
#     'learning_rate':0.001,
#     'metric':'binary_logloss'}
param = {'boosting_type':'gbdt',
                         'objective' : 'binary', #
                         #'metric' : 'binary_logloss',
                         'metric' : 'auc',
#                          'metric' : 'self_metric',
                         'learning_rate' : 0.01,
                         'max_depth' : 15,
                         'feature_fraction':0.8,
                         'bagging_fraction': 0.9,
                         'bagging_freq': 8,
                         'lambda_l1': 0.6,
                         'lambda_l2': 0,
#                          'scale_pos_weight':k,
#                         'is_unbalance':True
        }

In [69]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)


NameError: name 'lgb' is not defined

In [71]:
model = lgb.train(param,train_data,valid_sets=[train_data,valid_data],num_boost_round = 10000 ,early_stopping_rounds=200,verbose_eval=25, categorical_feature=attr)
predict=model.predict(test)
#print(predict)
test['Attrition']=predict
test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test['Attrition']=predict
# test[['Attrition']].to_csv('submit_lgb.csv')

NameError: name 'lgb' is not defined

In [72]:
### Attrition LR

In [73]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

In [74]:
train=pd.read_csv('../../resource/BI/train.csv',index_col=0)
test=pd.read_csv('../../resource/BI/test.csv',index_col=0)

#print(train['Attrition'].value_counts())


In [75]:
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [76]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [77]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)
# train.to_csv('train_label_encoder.csv')


In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [82]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [83]:
model = LogisticRegression(max_iter=100, 
                           verbose=True, 
                           random_state=33,
                           tol=1e-4
                          )

In [84]:
model.fit(X_train, y_train)
predict = model.predict_proba(test)[:, 1]
test['Attrition']=predict

print(test['Attrition'])
# test[['Attrition']].to_csv('submit_lr.csv')
print('submit_lr.csv saved')
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test[['Attrition']].to_csv('submit_lr.csv')

user_id
442     0.107272
1091    0.328287
981     0.240421
785     0.072643
1332    0.350858
          ...   
1439    0.059847
481     0.152947
124     0.139410
198     0.172930
1229    0.221405
Name: Attrition, Length: 294, dtype: float64
submit_lr.csv saved


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/du/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [85]:
## Attrition NGBoost

In [86]:
import pandas as pd

In [87]:
train=pd.read_csv('../../resource/BI/train.csv',index_col=0)
test=pd.read_csv('../../resource/BI/test.csv',index_col=0)

In [88]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

In [89]:
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())


In [90]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)


In [91]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

In [93]:
import ngboost as ng
import numpy as np
from sklearn.model_selection import train_test_split

In [94]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [95]:
model = ng.NGBClassifier(n_estimators=1000, 
                              learning_rate=0.01, 
                              verbose=True, 
                              verbose_eval=50
                             )


model.fit(X_train, y_train)


[iter 0] loss=0.4190 val_loss=0.0000 scale=1.0000 norm=2.0000
[iter 50] loss=0.3427 val_loss=0.0000 scale=1.0000 norm=1.7212
[iter 100] loss=0.3005 val_loss=0.0000 scale=1.0000 norm=1.6308
[iter 150] loss=0.2782 val_loss=0.0000 scale=1.0000 norm=1.5989
[iter 200] loss=0.2683 val_loss=0.0000 scale=1.0000 norm=1.5975
[iter 250] loss=0.2606 val_loss=0.0000 scale=1.0000 norm=1.5968
[iter 300] loss=0.2531 val_loss=0.0000 scale=1.0000 norm=1.5853
[iter 350] loss=0.2493 val_loss=0.0000 scale=1.0000 norm=1.5803
[iter 400] loss=0.2465 val_loss=0.0000 scale=0.5000 norm=0.7870
[iter 450] loss=0.2442 val_loss=0.0000 scale=0.1250 norm=0.1966
[iter 500] loss=0.2421 val_loss=0.0000 scale=1.0000 norm=1.5711
[iter 550] loss=0.2407 val_loss=0.0000 scale=0.0002 norm=0.0004
[iter 600] loss=0.2407 val_loss=0.0000 scale=0.0002 norm=0.0004
[iter 650] loss=0.2407 val_loss=0.0000 scale=0.0002 norm=0.0004
[iter 700] loss=0.2407 val_loss=0.0000 scale=0.0010 norm=0.0015
[iter 750] loss=0.2402 val_loss=0.0000 scal

NGBClassifier(n_estimators=1000,
              random_state=RandomState(MT19937) at 0x7FDC80BCEE20,
              verbose_eval=50)

In [97]:
#predict = model.predict(test)
predict = model.predict_proba(test)[:, 1]
#print(predict)
test['Attrition']=predict
## 转化为二分类输出
test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test[['Attrition']].to_csv('submit_ngb.csv')


In [98]:
### Attrition SVC

In [99]:
import pandas as pd

In [110]:
train=pd.read_csv('../../resource/BI/train.csv',index_col=0)
test=test1=pd.read_csv('../../resource/BI/test.csv',index_col=0)

In [111]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

In [112]:
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [113]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)


In [114]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

In [115]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [116]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)


mms = MinMaxScaler(feature_range=(0, 1))
X_train = mms.fit_transform(X_train)
X_valid = mms.fit_transform(X_valid)
test = mms.fit_transform(test)

In [117]:
model = SVC(kernel='rbf', 
			gamma="auto",
			max_iter=1000,
			random_state=33,
			verbose=True,
			tol=1e-5,
			cache_size=50000
		   )
#print(X_train)
#print(y_train)
#print(sum(y_train))

model = LinearSVC(
			max_iter=1000,
			random_state=33,
			verbose=True,
		   )
model.fit(X_train, y_train)
predict = model.predict(test)
print(predict)
#print(test)
#predict = model.predict_proba(test)[:, 1]
test1['Attrition']=predict

[LibLinear][0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]


In [109]:
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test1[['Attrition']].to_csv('submit_svc.csv')

In [118]:
### Attrition XGBoost

In [119]:
import pandas as pd

In [120]:
train=pd.read_csv('../../resource/BI/train.csv',index_col=0)
test=pd.read_csv('../../resource/BI/test.csv',index_col=0)

In [121]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

In [122]:
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [123]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [124]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('temp.csv')
#print(train)

In [125]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [126]:
param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', #
                         'eval_metric' : 'auc',
                         'eta' : 0.01,
                         'max_depth' : 15,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,
}

In [127]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [128]:
train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)
test_data = xgb.DMatrix(test)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test_data)
test['Attrition']=predict
print(predict)

[00:26:30] WARNING: /private/var/folders/h4/7cr1cmpn7v5b3x20_9wz8m740000gn/T/pip-req-build-nxocilc1/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:573: 
Parameters: { "boosting_type", "subsample_freq" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.84716	valid-auc:0.68455
[25]	train-auc:0.96263	valid-auc:0.76094
[50]	train-auc:0.97222	valid-auc:0.77431
[75]	train-auc:0.98666	valid-auc:0.77769
[100]	train-auc:0.99135	valid-auc:0.77606
[125]	train-auc:0.99367	valid-auc:0.78097
[150]	train-auc:0.99584	valid-auc:0.78555
[175]	train-auc:0.99762	valid-auc:0.78937
[200]	train-auc:0.99852	valid-auc:0.79515
[225]	train-auc:0.99900	valid-auc:0.80007
[250]	train-auc:0.99930	valid-auc:0.80214
[275]	train-auc:0.99968	valid-auc:0.80127
[300]	train-auc:0.99983	valid

In [129]:
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test[['Attrition']].to_csv('submit_xgb.csv')

In [130]:
## Manage